# Loan Application Orchestration Pipeline Demo (Agent Orchestration)

This notebook demonstrates the full loan application decision process **using the agent orchestration pipeline**. The orchestration function calls all tools and combines the results into a final summary.

In [13]:
# Import required modules and orchestration pipeline
from data_model import LoanApplicationJourney
from utils.config_loader import ConfigLoader
from orchestrator_pipeline import orchestrate_application
from tools.synthesize_summary import synthesize_summary
from agents import RunContextWrapper

In [35]:
# Create a test LoanApplicationJourney instance
test_app = LoanApplicationJourney(
    application_id="APP-LOCAL-001",
    submitted_time="2025-06-01T09:00:00",
    reviewed_time="2025-06-01T09:30:00",
    approved_time="2025-06-01T10:00:00",
    rejected_time=None,
    processing_steps={"KYC": 72, "CreditCheck": 50, "FinalApproval": 35},
    flagged_for_fraud=False,
    monthly_income=50000,
    monthly_costs=1000,
    requested_amount=25000,
    monthly_debt=400
)

In [36]:
# Run the agent orchestration pipeline
config = ConfigLoader()
report = orchestrate_application(test_app, config=config)
summary = synthesize_summary(RunContextWrapper(report))

Matching with: affordability=approve, fraud=False, sla=True


In [37]:
summary

{'application_id': 'APP-LOCAL-001',
 'sla_result': 'SLA violations in: KYC, CreditCheck, FinalApproval',
 'fraud_result': 'No fraud indicators detected',
 'affordability_result': '✅ Good affordability',
 'affordability_level': 'approve',
 'recommendation': '✅ Audit trail valid and sequential | ✅ Good affordability',
 'explanation': 'At least one process exceeded SLA limit.',
 'timestamp': '2025-06-28T17:04:11.853585',
 'final_decision': 'Undetermined',
 'explanation_sources': {},
 'summary_text': '✉️ **Loan Decision Summary**\n- Application ID: APP-LOCAL-001\n- Final Decision: Undetermined\n- SLA Result: SLA violations in: KYC, CreditCheck, FinalApproval\n- Fraud Result: No fraud indicators detected\n- Affordability: ✅ Good affordability (approve)\n- Recommendation: ✅ Audit trail valid and sequential | ✅ Good affordability\n- Explanation: At least one process exceeded SLA limit.\n- Timestamp: 2025-06-28T17:04:11.853585'}

## Integrate and demonstrate the agents

In [28]:
from dotenv import load_dotenv
load_dotenv("/Users/luca.dicarlo/.secrets_file", override=True)

True

In [29]:
# Import agent objects
from agents.fraud_agent import fraud_agent
from agents.sla_agent import sla_agent
from agents.recommendation_agent import recommendation_agent
from agents.orchestrator_agent import orchestrator_agent
from agents.report_agent import report_agent

In [30]:
# Example: Use fraud_agent to check for fraud indicators (sync call for demo)
fraud_result = fraud_agent.tools[0](RunContextWrapper(test_app))
print('Fraud Agent Result:', fraud_result)

Fraud Agent Result: {'flagged': False, 'label': 'No fraud indicators detected', 'risk_score': 0.0, 'triggered_rules': [], 'explanation': 'No indicators present'}


In [21]:
# Example: Use sla_agent to check SLA compliance (sync call for demo)
sla_result = sla_agent.tools[0](RunContextWrapper(test_app))
print('SLA Agent Result:', sla_result)

SLA Agent Result: {'violated': True, 'label': 'SLA violations in: KYC', 'violated_steps': ['KYC'], 'explanation': 'Steps exceeded configured SLA limits.'}


In [22]:
# Example: Use recommendation_agent to get a recommendation (sync call for demo)
rec_result = recommendation_agent.run('Provide recommendation', context=test_app)
print('Recommendation Agent Result:', getattr(rec_result, 'final_output', rec_result))

Recommendation Agent Result: <coroutine object Agent.run at 0x7f98c81e6b20>


In [23]:
# Example: Use orchestrator_agent to run the full orchestration (sync call for demo)
orc_result = orchestrator_agent.tools[0](RunContextWrapper(test_app))
print('Orchestrator Agent Result:', orc_result)

Orchestrator Agent Result: {'violated': True, 'label': 'SLA violations in: KYC', 'violated_steps': ['KYC'], 'explanation': 'Steps exceeded configured SLA limits.'}


In [24]:
# Example: Use report_agent to synthesize the summary (sync call for demo)
rep_result = report_agent.tools[0](RunContextWrapper(report))
print('Report Agent Result:', rep_result)

Report Agent Result: {'application_id': 'APP-LOCAL-001', 'sla_result': 'SLA violations in: KYC, FinalApproval', 'fraud_result': 'No fraud indicators detected', 'affordability_result': '❌ Low affordability score', 'affordability_level': 'reject', 'recommendation': '✅ Audit trail valid and sequential | ❌ Low affordability score', 'explanation': 'Income to debt ratio exceeds safe threshold. At least one process exceeded SLA limit.', 'timestamp': '2025-06-28T15:35:08.529469', 'final_decision': '❌ Reject due to affordability and delay', 'explanation_sources': {}, 'summary_text': '✉️ **Loan Decision Summary**\n- Application ID: APP-LOCAL-001\n- Final Decision: ❌ Reject due to affordability and delay\n- SLA Result: SLA violations in: KYC, FinalApproval\n- Fraud Result: No fraud indicators detected\n- Affordability: ❌ Low affordability score (reject)\n- Recommendation: ✅ Audit trail valid and sequential | ❌ Low affordability score\n- Explanation: Income to debt ratio exceeds safe threshold. At